In [1]:
import hopsworks
import pandas as pd
import json

## Creating a feature group from the feature group with Json Data

#### Login to Hopsworks and fetch feature group

In [2]:
# Login to Hopsworks.
project = hopsworks.login()

# Fetch the feature store.
fs = project.get_feature_store()

# Fetch the feature group
fg_json = fs.get_feature_group(name="fg_raw_event_data", version=1)

2025-03-12 22:55:41,405 INFO: Initializing external client
2025-03-12 22:55:41,405 INFO: Base URL: https://10.87.41.143:28181
2025-03-12 22:55:41,976 INFO: Python Engine initialized.

Logged in to project, explore it here https://10.87.41.143:28181/p/119


#### Read data from the feature group and perform required feature enginerring

In [3]:
# Read from feature group
df = fg_json.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.25s) 


In [4]:
# Extract the json data convert it into a dataframe with required columns
unnested_dataframe = pd.json_normalize(df["data"].apply(lambda x : json.loads(x)))
unnested_dataframe.head(2)

,event_time,event_id,user_id,click_count,time_spent,scroll_depth,purchase_amount,ad_interaction
0,2025-03-12T21:48:40.177399,1,24,17,3.06,42.19,14.90,NaN
1,2025-03-12T21:48:40.177770,17,11,6,19.84,39.56,457.27,NaN


In [5]:
# Perform required feature enginering
unnested_dataframe = unnested_dataframe[["event_time", "event_id", "user_id", "click_count", "time_spent", "scroll_depth", "purchase_amount"]]
# Convert string datetime to datetime object
unnested_dataframe["event_time"] = pd.to_datetime(unnested_dataframe["event_time"])

user_event_df = unnested_dataframe[["event_time", "event_id", "user_id", "purchase_amount"]]

events_df = unnested_dataframe[["event_time", "event_id", "click_count", "time_spent", "scroll_depth"]]


#### Create feature groups

There are two feature groups being created.
1. **An user-events feature group** : This feature group store all events for an user. The feature group has the primary key as `user_id` hence the online feature store will only contain the latest events for the user and the offline feature group will contain all event triggered by the user.
2. **An events feature group**: This feature group will conatin all information regarding the event. This feature group will have the primary key as `event_id`. Hence can be joined with the user-events feature group to create the entire dataframe.

Splitting the data like this also allows creating a seperate *users feature group* which could contain user specific details which can again be joined to create a feature view.

In [6]:
# Creating user-events feature group
fg_user_events = fs.get_or_create_feature_group(name = "fg_user_events",
                                                version = 1, 
                                                primary_key = ["user_id"],
                                                event_time = ["event_time"],
                                                online_enabled=True)
# Creating events feature group
fg_events = fs.get_or_create_feature_group(name = "fg_events",
                                                version = 1, 
                                                primary_key = ["event_id"],
                                                event_time = ["event_time"],
                                                online_enabled=True)


# Inserting data into the feature groups
fg_user_events.insert(user_event_df)
fg_events.insert(events_df)

2025-03-12 22:55:44,874 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.

Feature Group created successfully, explore it at 
https://10.87.41.143:28181/p/119/fs/67/fg/32


Uploading Dataframe: 100.00% |█| Rows 40/40 | Elapsed Time: 00:00 | Remaining Ti


Launching job: fg_user_events_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://10.87.41.143:28181/p/119/jobs/named/fg_user_events_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://10.87.41.143:28181/p/119/fs/67/fg/33


Uploading Dataframe: 100.00% |█| Rows 40/40 | Elapsed Time: 00:00 | Remaining Ti


Launching job: fg_events_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://10.87.41.143:28181/p/119/jobs/named/fg_events_1_offline_fg_materialization/executions


(Job('fg_events_1_offline_fg_materialization', 'SPARK'), None)

## Creating a feature view and generating train-test data

#### Define a query to join feature groups

The joins performed by Hopsworks are always point in time correct base on event time. Hence you can easily join the the user-events and the users feature groups to create a new feature view that has point int time correct data.

In [8]:
query = fg_user_events.select("purchase_amount").join(fg_events.select_features(), prefix="event_data_", on="event_id")
query.show(5)

2025-03-12 22:57:49,904 INFO: Using ['click_count', 'time_spent', 'scroll_depth'] as features for the query.To include primary key and event time use `select_all`.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 


,purchase_amount,event_data_click_count,event_data_time_spent,event_data_scroll_depth
0,51.25,8,7.72,13.95
1,177.64,20,25.11,16.27
2,438.18,2,6.72,94.29
3,304.49,20,9.30,63.89
4,157.73,1,7.26,3.21


#### Creating feature view

In [9]:
# Import any require model-dependent transformation functions
from hopsworks.hsfs.builtin_transformations import min_max_scaler

# Create feature view
fv = fs.get_or_create_feature_view(name="fv_events", 
                                   version = 1, 
                                   query = query, 
                                   transformation_functions=[
                                       min_max_scaler("event_data_click_count"), 
                                       min_max_scaler("event_data_time_spent"), 
                                       min_max_scaler("event_data_scroll_depth")
                                   ],
                                   labels = ["purchase_amount"])


# Create training data
X_train, X_test, y_train, y_test = fv.train_test_split(test_size = 0.2)

Feature view created successfully, explore it at 
https://10.87.41.143:28181/p/119/fs/67/fv/fv_events/version/1
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.48s) 
2025-03-12 22:57:59,221 WARNING: VersionWarning: Incremented version to `1`.



## Testing a new feature

You can easily test a new fetaure without disturbing existing feature groups by creating a seperate feature group with only the new tests feature, this feature group can be joined with existing feature groups to create a new feature view. Once testing is done, the new feature can then be appended to an existing feature group can be used to create and backfill an new feature group.

In [10]:
# Fetch the feature group with the raw json
fg_json = fs.get_feature_group(name="fg_raw_event_data", version=1)

# Read the data from the raw feature group.
df = fg_json.read()
unnested_dataframe = pd.json_normalize(df["data"].apply(lambda x : json.loads(x)))
unnested_dataframe["event_time"] = pd.to_datetime(unnested_dataframe["event_time"])

# Extract the new feature that is required to be tested.
test_df = unnested_dataframe[["event_time", "event_id", "ad_interaction"]]

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.24s) 


In [11]:
# Create the new test feature_group
fg_test = fs.get_or_create_feature_group(name = "fg_test",
                                                version = 1, 
                                                primary_key = ["event_id"],
                                                event_time = ["event_time"],
                                                online_enabled=True)

fg_test.insert(test_df)

2025-03-12 22:58:00,683 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.

Feature Group created successfully, explore it at 
https://10.87.41.143:28181/p/119/fs/67/fg/34


Uploading Dataframe: 100.00% |█| Rows 40/40 | Elapsed Time: 00:00 | Remaining Ti


Launching job: fg_test_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://10.87.41.143:28181/p/119/jobs/named/fg_test_1_offline_fg_materialization/executions


(Job('fg_test_1_offline_fg_materialization', 'SPARK'), None)

In [13]:
# Create a new testing feature view that joins the feature groups : fg_user_events, fg_events and fg_test
query = fg_user_events.select("purchase_amount").join(fg_events.select_features(), prefix="event_data_", on="event_id").join(fg_test.select_features(), prefix="testing_", on="event_id")
query.show(5)

2025-03-12 22:59:49,073 INFO: Using ['click_count', 'time_spent', 'scroll_depth'] as features for the query.To include primary key and event time use `select_all`.
2025-03-12 22:59:49,075 INFO: Using ['ad_interaction'] as features for the query.To include primary key and event time use `select_all`.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 


,purchase_amount,event_data_click_count,event_data_time_spent,event_data_scroll_depth,testing_ad_interaction
0,51.25,8,7.72,13.95,NaN
1,177.64,20,25.11,16.27,NaN
2,438.18,2,6.72,94.29,NaN
3,304.49,20,9.30,63.89,1.0
4,157.73,1,7.26,3.21,NaN


In [14]:
# Create testing feature view
fv_test = fs.get_or_create_feature_view(name="fv_test", 
                                   version = 1, 
                                   query = query, 
                                   transformation_functions=[
                                       min_max_scaler("event_data_click_count"), 
                                       min_max_scaler("event_data_time_spent"), 
                                       min_max_scaler("event_data_scroll_depth"),
                                       min_max_scaler("testing_ad_interaction")
                                   ],
                                   labels = ["purchase_amount"])

X_train, X_test, y_train, y_test = fv_test.train_test_split(test_size = 0.2)

Feature view created successfully, explore it at 
https://10.87.41.143:28181/p/119/fs/67/fv/fv_test/version/1
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.53s) 
2025-03-12 23:00:01,453 WARNING: VersionWarning: Incremented version to `1`.



### Appending new features to the feature view
Once a feature has been testing and wants to be included in a feature group it can be done in two ways 
1. Appending the feature to an existing feature group and backfilling the data.
2. Creating a new version of the feature group that contains the new feature and backfilling the data.

#### Option 1: Appending the feature to the fetaure group

##### Appending the feature

In [17]:
# Create the feature to be appended
from hsfs.feature import Feature
features = [Feature(name="ad_interaction",type="double",online_type="double")]

In [18]:
# Appending a new feature to the feature group
fg_events.append_features(features)

2025-03-12 23:00:21,532 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-03-12 23:00:24,614 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-03-12 23:01:26,270 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-03-12 23:01:26,352 INFO: Waiting for log aggregation to finish.
2025-03-12 23:01:34,771 INFO: Execution finished successfully.


##### Backfilling data to the feature group

In [19]:
# Preparing data to be backfilled
df = fg_json.read()
unnested_dataframe = pd.json_normalize(df["data"].apply(lambda x : json.loads(x)))
unnested_dataframe["event_time"] = pd.to_datetime(unnested_dataframe["event_time"])

events_df = unnested_dataframe[["event_time", "event_id", "click_count", "time_spent", "scroll_depth", "ad_interaction"]]

# Insert the data into the feature group
fg_events.insert(events_df)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.29s) 


Uploading Dataframe: 100.00% |█| Rows 40/40 | Elapsed Time: 00:00 | Remaining Ti


Launching job: fg_events_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://10.87.41.143:28181/p/119/jobs/named/fg_events_1_offline_fg_materialization/executions


(Job('fg_events_1_offline_fg_materialization', 'SPARK'), None)

#### Option 2: Creating the new feature group version

##### Creating a new version of the feature group

In [20]:
# Creating events feature group
fg_events_v2 = fs.get_or_create_feature_group(name = "fg_events",
                                                version = 2, 
                                                primary_key = ["event_id"],
                                                event_time = ["event_time"],
                                                online_enabled=True)


2025-03-12 23:01:56,339 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



##### Backfilling data to the feature group

In [21]:
# Preparing data to be backfilled
df = fg_json.read()
unnested_dataframe = pd.json_normalize(df["data"].apply(lambda x : json.loads(x)))
unnested_dataframe["event_time"] = pd.to_datetime(unnested_dataframe["event_time"])

events_df = unnested_dataframe[["event_time", "event_id", "click_count", "time_spent", "scroll_depth", "ad_interaction"]]

# Insert the data into the feature group
fg_events_v2.insert(events_df)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.24s) 
Feature Group created successfully, explore it at 
https://10.87.41.143:28181/p/119/fs/67/fg/35


Uploading Dataframe: 100.00% |█| Rows 40/40 | Elapsed Time: 00:00 | Remaining Ti


Launching job: fg_events_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://10.87.41.143:28181/p/119/jobs/named/fg_events_2_offline_fg_materialization/executions


(Job('fg_events_2_offline_fg_materialization', 'SPARK'), None)